In [58]:
from pathlib import Path
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from IPython.display import Markdown as md


image_folder = Path("test_images")
image_paths = [path for path in image_folder.iterdir()]

#image_path = Path("test_images/20230713-105000.jpg")
image_path = Path("test_images/20230714-060937.jpg")
#image_path = Path("test_images/20230714-071015.jpg")
image_path.exists()

True

In [56]:
# HELPER FUNCTIONS
from typing import List


def rgb2hex(rgb):
    hex = "#{:02x}{:02x}{:02x}".format(int(rgb[0]), int(rgb[1]), int(rgb[2]))
    return hex


def hex2rgb(h):
    h = h.lstrip('#')
    return [int(h[i:i + 2], 16) for i in (0, 2, 4)]

def calculate_new_size(image, WIDTH, HEIGHT):
    if image.width >= image.height:
        wpercent = (WIDTH / float(image.width))
        hsize = int((float(image.height) * float(wpercent)))
        new_width, new_height = WIDTH, hsize
    else:
        hpercent = (HEIGHT / float(image.height))
        wsize = int((float(image.width) * float(hpercent)))
        new_width, new_height = wsize, HEIGHT
    return new_width, new_height

def print_color(color: str):
    return md(f"<div style='background-color: {color}; min-height: 50px'></div>")

def print_image_info(image: Image) -> None:
    print(
        "Loaded {f} image {n}. Size: {s:.2f} KB. Dimensions: ({d})".format(
            f=image.format,
            n=image_path.name,
            d=image.size,
        )
    )


DIV_TEMPLATE = "<div style='background-color: {color}; min-height: 50px'>{color}</div>"

def print_colors(colors: List[str]) -> None:
    return md("".join([DIV_TEMPLATE.format(color=color) for color in colors]))

In [37]:
def dominant_color_via_kmean(image_path: Path) -> str:
    image = Image.open(image_path)

    # Transform image
    new_width, new_height = calculate_new_size(image, 128, 128)
    image = image.resize((new_width, new_height), Image.ANTIALIAS)

    img_array = np.array(image)
    img_vector = img_array.reshape((img_array.shape[0] * img_array.shape[1], 3))

    # Modeling
    model = KMeans(n_clusters=6)
    labels = model.fit_predict(img_vector)
    label_counts = Counter(labels)

    hex_colors = [rgb2hex(center) for center in model.cluster_centers_]

    results = list(zip(hex_colors, list(label_counts.values())))
    results.sort(key=lambda tup: tup[1], reverse=True)


    hex_color, count = results[0]
    return hex_color

In [38]:
from colorthief import ColorThief

def dominant_color_via_color_thief(image_path: Path) -> str:
    color_thief = ColorThief(image_path)
    # get the dominant color
    dominant_color = color_thief.get_color(quality=10)
    return rgb2hex(dominant_color)


In [63]:
import scipy.cluster
import sklearn.cluster

def dominant_color_via_mini_kmeans(image_path: Path) -> str:
    image = Image.open(image_path)

    image = image.resize((128, 128))      # optional, to reduce time
    ar = np.asarray(image)
    shape = ar.shape
    ar = ar.reshape(np.product(shape[:2]), shape[2]).astype(float)

    kmeans = sklearn.cluster.MiniBatchKMeans(
        n_clusters=6,
        init="k-means++",
        max_iter=20,
        random_state=1000
    ).fit(ar)
    codes = kmeans.cluster_centers_

    vecs, _dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
    counts, _bins = np.histogram(vecs, len(codes))    # count occurrences

    colors = []
    for index in np.argsort(counts)[::-1]:
        colors.append(tuple([int(code) for code in codes[index]]))
    return rgb2hex(colors[0])

In [64]:
import time


functions = [
    dominant_color_via_kmean,
    dominant_color_via_mini_kmeans,
    # dominant_color_via_color_thief
]

colors = []
for image_path in image_paths:
    for fct in functions:
        print(f"Testing function: {fct.__name__}")

        start_time = time.time()
        colors.append(fct(image_path))

        print(f"Execution time: {time.time() - start_time}\n")

    colors.append("#000")

print_colors(colors)

Testing function: dominant_color_via_kmean
Execution time: 2.7150888442993164

Testing function: dominant_color_via_mini_kmeans
Execution time: 0.2840602397918701

Testing function: dominant_color_via_kmean
Execution time: 3.3999946117401123

Testing function: dominant_color_via_mini_kmeans
Execution time: 0.30806756019592285

Testing function: dominant_color_via_kmean
Execution time: 2.696955442428589

Testing function: dominant_color_via_mini_kmeans
Execution time: 0.27497267723083496

Testing function: dominant_color_via_kmean
Execution time: 2.3656318187713623

Testing function: dominant_color_via_mini_kmeans
Execution time: 0.2404782772064209

Testing function: dominant_color_via_kmean
Execution time: 2.6068825721740723

Testing function: dominant_color_via_mini_kmeans
Execution time: 0.2406773567199707

Testing function: dominant_color_via_kmean
Execution time: 2.4745805263519287

Testing function: dominant_color_via_mini_kmeans
Execution time: 0.27553319931030273



<div style='background-color: #3e281c; min-height: 50px'>#3e281c</div><div style='background-color: #a8925f; min-height: 50px'>#a8925f</div><div style='background-color: #000; min-height: 50px'>#000</div><div style='background-color: #bb875b; min-height: 50px'>#bb875b</div><div style='background-color: #ba885c; min-height: 50px'>#ba885c</div><div style='background-color: #000; min-height: 50px'>#000</div><div style='background-color: #375a72; min-height: 50px'>#375a72</div><div style='background-color: #34546b; min-height: 50px'>#34546b</div><div style='background-color: #000; min-height: 50px'>#000</div><div style='background-color: #60badd; min-height: 50px'>#60badd</div><div style='background-color: #107eb3; min-height: 50px'>#107eb3</div><div style='background-color: #000; min-height: 50px'>#000</div><div style='background-color: #49512f; min-height: 50px'>#49512f</div><div style='background-color: #8eadd1; min-height: 50px'>#8eadd1</div><div style='background-color: #000; min-height: 50px'>#000</div><div style='background-color: #9e9e9f; min-height: 50px'>#9e9e9f</div><div style='background-color: #736455; min-height: 50px'>#736455</div><div style='background-color: #000; min-height: 50px'>#000</div>